In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pmdarima as pm
from pmdarima import auto_arima

"""
pd.set_option('display.max_rows', None)   # Shows all rows in the DataFrame
pd.set_option('display.max_columns', None)  # Shows all columns in the DataFrame
pd.set_option('display.width', None)     # Ensures no line breaks for wide DataFrames
pd.set_option('display.max_colwidth', None)
"""

"\npd.set_option('display.max_rows', None)   # Shows all rows in the DataFrame\npd.set_option('display.max_columns', None)  # Shows all columns in the DataFrame\npd.set_option('display.width', None)     # Ensures no line breaks for wide DataFrames\npd.set_option('display.max_colwidth', None)\n"

# Import Data

In [20]:
df = pd.read_csv('data/SAP Datasets.csv')

# forward and backward filling
years_col = [str(i) for i in range(2000,2024)]
df[years_col] = df[years_col].apply(lambda row: row.ffill().bfill(), axis=1)

# drop unnecessary columns
df = df.drop(['Country Code', 'short description', 'long description', 'Indicator Code'], axis=1)
df



/var/folders/ds/b8dd7ztn06l6p5dfvy90q01c0000gn/T/ipykernel_12913/4202544317.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/SAP Datasets.csv')


,Country Name,Indicator Name,Topic,Unit of measure,2000,2001,2002,2003,2004,2005,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Aruba,Control of Corruption: Estimate,Public Sector: Policy & institutions,0,1.165965,1.165965,1.165965,1.165965,1.165965,1.269663,...,1.014232,1.248461,1.232735,1.239377,1.207429,1.174703,1.199392,0.801611,0.756785,0.756785
1,Africa Eastern and Southern,Control of Corruption: Estimate,Public Sector: Policy & institutions,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,Control of Corruption: Estimate,Public Sector: Policy & institutions,0,-1.271724,-1.271724,-1.251137,-1.344180,-1.350647,-1.447252,...,-1.364743,-1.354240,-1.540353,-1.531910,-1.502881,-1.419499,-1.493700,-1.152327,-1.183776,-1.183776
3,Africa Western and Central,Control of Corruption: Estimate,Public Sector: Policy & institutions,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,Control of Corruption: Estimate,Public Sector: Policy & institutions,0,-1.197514,-1.197514,-1.155493,-1.335463,-1.377569,-1.323182,...,-1.457798,-1.427323,-1.483337,-1.444333,-1.199251,-1.058418,-0.938673,-0.651610,-0.601941,-0.601941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23136,Kosovo,Urban population (% of total population),Environment: Density & urbanization,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23137,"Yemen, Rep.",Urban population (% of total population),Environment: Density & urbanization,0,26.267000,26.787000,27.315000,27.849000,28.390000,28.936000,...,34.165000,34.777000,35.394000,36.016000,36.642000,37.273000,37.908000,38.546000,39.188000,39.831000
23138,South Africa,Urban population (% of total population),Environment: Density & urbanization,0,56.891000,57.368000,57.898000,58.446000,58.993000,59.536000,...,64.312000,64.828000,65.341000,65.850000,66.355000,66.856000,67.354000,67.847000,68.335000,68.819000
23139,Zambia,Urban population (% of total population),Environment: Density & urbanization,0,34.802000,35.002000,35.475000,35.951000,36.430000,36.911000,...,41.382000,41.907000,42.438000,42.976000,43.521000,44.072000,44.629000,45.192000,45.761000,46.335000


# Ranking by Indicators

In [21]:
Indicators = pd.read_csv('data/Indicators_final.csv')

Indicators = Indicators[Indicators['DOES A HIGHER VALUE INCREASE POVERTY?'] != 'TBD']
Indicators


,INDICATOR NAMES,DOES A HIGHER VALUE INCREASE POVERTY?,Short Description,Long Description
0,Control of Corruption: Estimate,FALSE,NaN,NaN
1,Multilateral debt service (% of public and pub...,TRUE,Multilateral debt service is the repayment of ...,Multilateral debt service is the repayment of ...
2,Access to clean fuels and technologies for coo...,FALSE,NaN,Access to clean fuels and technologies for coo...
3,Access to clean fuels and technologies for coo...,FALSE,NaN,Access to clean fuels and technologies for coo...
4,Access to clean fuels and technologies for coo...,FALSE,NaN,Access to clean fuels and technologies for coo...
...,...,...,...,...
82,Teenage mothers (% of women ages 15-19 who hav...,TRUE,NaN,Teenage mothers are the percentage of women ag...
83,Age dependency ratio (% of working-age populat...,TRUE,NaN,Age dependency ratio is the ratio of dependent...
84,"Age dependency ratio, old (% of working-age po...",TRUE,NaN,"Age dependency ratio, old, is the ratio of old..."
85,"Age dependency ratio, young (% of working-age ...",TRUE,NaN,"Age dependency ratio, young, is the ratio of y..."


## Indicator Dataframes

In [22]:
results = {}

for index, row in Indicators.iterrows():
    indicator_name = row['INDICATOR NAMES']
    increase_poverty = row['DOES A HIGHER VALUE INCREASE POVERTY?']
    
    indicator_df = df[df['Indicator Name'] == indicator_name]
    indicator_df.set_index('Country Name', inplace=True)
    
    years_columns = [str(year) for year in range(2000, 2024)]
    country_year_data = indicator_df[years_columns]
    
    if increase_poverty == 'TRUE':
        direction = True  # Lower values get better ranks
    elif increase_poverty == 'FALSE':
        direction = False  # Higher values get better ranks
    
    ranked_countries = country_year_data.rank(axis=0, method='min', ascending=direction, na_option='keep')
    
    column_name = f"Average_Rank_{indicator_name}"
    ranked_countries[column_name] = ranked_countries.mean(axis=1, skipna=True)
    
    results[column_name] = ranked_countries[[column_name]].sort_values(by='Country Name').reset_index()



## Total Average

In [13]:
results['Average_Rank_Multilateral debt service (% of public and publicly guaranteed debt service)']['Country Name']

,Country Name,Average_Rank_Multilateral debt service (% of public and publicly guaranteed debt service)
0,Afghanistan,132.750000
1,Africa Eastern and Southern,NaN
2,Africa Western and Central,NaN
3,Albania,76.166667
4,Algeria,27.791667
...,...,...
261,West Bank and Gaza,NaN
262,World,NaN
263,"Yemen, Rep.",110.083333
264,Zambia,48.166667


In [14]:
results['Average_Rank_Control of Corruption: Estimate']

,Country Name,Average_Rank_Control of Corruption: Estimate
0,Afghanistan,195.916667
1,Africa Eastern and Southern,NaN
2,Africa Western and Central,NaN
3,Albania,141.875000
4,Algeria,142.208333
...,...,...
261,West Bank and Gaza,113.041667
262,World,NaN
263,"Yemen, Rep.",185.875000
264,Zambia,131.458333


In [24]:
combined_df = pd.concat([df.drop('Country Name', axis=1) for _, df in results.items()], axis=1)
combined_df.insert(0,'Country Name', results['Average_Rank_Control of Corruption: Estimate']['Country Name'])

In [25]:
combined_df.head()

,Country Name,Average_Rank_Control of Corruption: Estimate,Average_Rank_Multilateral debt service (% of public and publicly guaranteed debt service),"Average_Rank_Access to clean fuels and technologies for cooking, rural (% of rural population)","Average_Rank_Access to clean fuels and technologies for cooking, urban (% of urban population)",Average_Rank_Access to clean fuels and technologies for cooking (% of population),"Average_Rank_Access to electricity, rural (% of rural population)","Average_Rank_Access to electricity, urban (% of urban population)",Average_Rank_Access to electricity (% of population),Average_Rank_Compensation of employees (current LCU),...,"Average_Rank_Children in employment, wage workers (% of children in employment, ages 7-14)","Average_Rank_Adolescent fertility rate (births per 1,000 women ages 15-19)",Average_Rank_Wanted fertility rate (births per woman),Average_Rank_Women who were first married by age 15 (% of women ages 20-24),Average_Rank_Women who were first married by age 18 (% of women ages 20-24),Average_Rank_Teenage mothers (% of women ages 15-19 who have had children or are currently pregnant),Average_Rank_Age dependency ratio (% of working-age population),"Average_Rank_Age dependency ratio, old (% of working-age population)","Average_Rank_Age dependency ratio, young (% of working-age population)",Average_Rank_Urban population (% of total population)
0,Afghanistan,195.916667,132.750000,180.833333,162.750000,182.125000,195.291667,189.541667,201.500000,60.666667,...,71.125000,237.291667,55.666667,109.041667,112.708333,26.291667,257.708333,17.583333,258.541667,242.958333
1,Africa Eastern and Southern,NaN,NaN,181.541667,186.166667,186.916667,226.208333,237.208333,238.541667,NaN,...,NaN,232.958333,NaN,NaN,NaN,NaN,233.791667,42.333333,232.000000,218.250000
2,Africa Western and Central,NaN,NaN,201.916667,202.541667,200.666667,220.625000,216.666667,218.125000,NaN,...,NaN,246.333333,70.541667,NaN,NaN,50.833333,241.291667,47.708333,241.541667,184.916667
3,Albania,141.875000,76.166667,127.541667,132.416667,131.333333,21.875000,81.416667,55.041667,64.875000,...,10.958333,60.875000,2.083333,23.375000,39.875000,2.250000,87.500000,200.000000,78.500000,144.458333
4,Algeria,142.208333,27.791667,71.625000,52.500000,70.541667,120.250000,95.875000,104.625000,NaN,...,62.875000,38.791667,NaN,1.000000,15.375000,NaN,118.875000,106.458333,145.291667,92.625000


In [26]:
combined_df.to_csv('country_initial_rankings.csv')

In [97]:
indicator_result_df = pd.read_csv('indicator_result_df.csv')

In [98]:
indicator_result_df

,Indicator Name,Country Count,Normalized Count
0,"Birth rate, crude (per 1,000 people)",265,1.000000
1,"Age dependency ratio, young (% of working-age population)",265,1.000000
2,"Age dependency ratio, old (% of working-age population)",265,1.000000
3,Age dependency ratio (% of working-age population),265,1.000000
4,"Adolescent fertility rate (births per 1,000 women ages 15-19)",265,1.000000
5,People using at least basic drinking water services (% of population),263,0.992424
6,Urban population (% of total population),263,0.992424
7,People using at least basic sanitation services (% of population),263,0.992424
8,Access to electricity (% of population),263,0.992424
9,"International migrant stock, total",261,0.984848


In [96]:
all_ranks['Overall Mean Rank'] = combined_df.mean(axis=1)

sorted_overall_ranks = all_ranks.sort_values(by='Overall Mean Rank')

print(sorted_overall_ranks['Overall Mean Rank'])

Country Name
Switzerland                                              41.931122
Singapore                                                42.750000
Netherlands                                              44.323980
Sint Maarten (Dutch part)                                44.901515
Iceland                                                  46.643707
Korea, Rep.                                              47.788012
Denmark                                                  48.164167
Malta                                                    48.604911
Australia                                                48.836667
Norway                                                   49.133170
Germany                                                  51.000833
Austria                                                  51.086735
Cayman Islands                                           51.400463
Gibraltar                                                52.520833
Belgium                                          